In [1]:
import cv2
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
from random import shuffle
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
import os 
import numpy as np
from moviepy.editor import VideoFileClip

Using TensorFlow backend.


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#path of the folder in which labelled bmis of different activities are stored
train_data='D:\\all'

In [4]:
#storing the image and label of train data

def train_data_with_label():
    train_images=[]
    for i in tqdm(os.listdir(train_data)):
        path=os.path.join(train_data,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        label=path.split('_')[1]
        if label=='walk.avi.jpg':
            ohl=np.array([1,0,0,0,0])
        if label=='jog.avi.jpg':
            ohl=np.array([0,1,0,0,0])
        if label=='side.avi.jpg':
            ohl=np.array([0,0,1,0,0])
        if label=='jump.avi.jpg':
            ohl=np.array([0,0,0,1,0])
        if label=='run.avi.jpg':
            ohl=np.array([0,0,0,0,1])
        train_images.append([np.array(img),ohl])
    shuffle(train_images)
    return train_images

training_images = train_data_with_label()

tr_img_data = np.array([i[0] for i in training_images]).reshape(-1,64,64,1)
tr_lbl_data = np.array([i[1] for i in training_images])




100%|█████████████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 100.13it/s]


In [7]:
#cnn model

model=Sequential()
model.add(InputLayer(input_shape=[64,64,1]))
model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))         
model.add(Conv2D(filters=50,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))         
model.add(Conv2D(filters=80,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))          
model.add(Dropout(0.25)) 
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(5,activation='softmax')) #5 labels
optimizer=Adam(lr=1e-3)

model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=tr_img_data,y=tr_lbl_data,epochs=25,batch_size=15)
model.summary()

Epoch 1/25
62/62 [==============================] - ETA: 5s - loss: 4.6756 - acc: 0.400 - ETA: 2s - loss: 5.0544 - acc: 0.400 - ETA: 1s - loss: 5.0164 - acc: 0.377 - ETA: 0s - loss: 5.1402 - acc: 0.316 - 4s 70ms/step - loss: 5.2419 - acc: 0.3226
Epoch 2/25
62/62 [==============================] - ETA: 2s - loss: 4.5884 - acc: 0.266 - ETA: 2s - loss: 4.5183 - acc: 0.266 - ETA: 1s - loss: 4.9030 - acc: 0.200 - ETA: 0s - loss: 4.7036 - acc: 0.200 - 4s 59ms/step - loss: 4.5973 - acc: 0.2097
Epoch 3/25
62/62 [==============================] - ETA: 1s - loss: 5.5428 - acc: 0.066 - ETA: 2s - loss: 4.0776 - acc: 0.300 - ETA: 1s - loss: 3.4357 - acc: 0.333 - ETA: 0s - loss: 3.0556 - acc: 0.316 - 4s 67ms/step - loss: 3.0284 - acc: 0.3226
Epoch 4/25
62/62 [==============================] - ETA: 2s - loss: 2.4264 - acc: 0.266 - ETA: 1s - loss: 2.4426 - acc: 0.200 - ETA: 0s - loss: 2.1767 - acc: 0.222 - ETA: 0s - loss: 2.0825 - acc: 0.216 - 4s 61ms/step - loss: 2.0682 - acc: 0.2097
Epoch 5/25
62/62

In [9]:
#path of the video to be tested
path="D:\\side\\denis_side.avi"
cam = cv2.VideoCapture(path) 

#generating bmi for the video to be tested
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
clip = VideoFileClip(path)

c=clip.duration*2
       

try: 

    if not os.path.exists('D:\\testbmi'): 
        os.makedirs('D:\\testbmi') 
  
except OSError: 
    print ('Error: Creating directory of data') 
   
currentframe = 0
blended=0
  
while(True): 
      
    # reading from frame 
    ret,frame = cam.read() 
 
    if ret: 
        # if video is still left continue creating images 
        mask = fgbg.apply(frame)
        
        mask = cv2.dilate(mask,None,iterations=2)
       
        name = 'D:\\testbmi\\mask' + str(currentframe) + '.jpg'
        print ('Creating...' + name) 
        
        #intensity function
        mask=mask/(c*c*c)
        
        c=c-0.04
         
        mask=cv2.resize(mask,(64,64))
        
        blended+=mask
  
        currentframe += 1
    else: 
        break
  
# Release all space and windows once done 
cam.release() 
cv2.destroyAllWindows()

#saving the generated bmi
cv2.imwrite("D:\\pic.jpg",blended)

Creating...D:\testbmi\mask0.jpg
Creating...D:\testbmi\mask1.jpg
Creating...D:\testbmi\mask2.jpg
Creating...D:\testbmi\mask3.jpg
Creating...D:\testbmi\mask4.jpg
Creating...D:\testbmi\mask5.jpg
Creating...D:\testbmi\mask6.jpg
Creating...D:\testbmi\mask7.jpg
Creating...D:\testbmi\mask8.jpg
Creating...D:\testbmi\mask9.jpg
Creating...D:\testbmi\mask10.jpg
Creating...D:\testbmi\mask11.jpg
Creating...D:\testbmi\mask12.jpg
Creating...D:\testbmi\mask13.jpg
Creating...D:\testbmi\mask14.jpg
Creating...D:\testbmi\mask15.jpg
Creating...D:\testbmi\mask16.jpg
Creating...D:\testbmi\mask17.jpg
Creating...D:\testbmi\mask18.jpg
Creating...D:\testbmi\mask19.jpg
Creating...D:\testbmi\mask20.jpg
Creating...D:\testbmi\mask21.jpg
Creating...D:\testbmi\mask22.jpg
Creating...D:\testbmi\mask23.jpg
Creating...D:\testbmi\mask24.jpg
Creating...D:\testbmi\mask25.jpg
Creating...D:\testbmi\mask26.jpg
Creating...D:\testbmi\mask27.jpg
Creating...D:\testbmi\mask28.jpg
Creating...D:\testbmi\mask29.jpg
Creating...D:\testbm

True

In [23]:
#feeding the bmi to the model and printing the accuracy
data=blended.reshape(1,64,64,1)
model_out=model.predict([data])
print(model_out)

if np.argmax(model_out)==0:
    label="walking"
if np.argmax(model_out)==1:
    label="jogging"
if np.argmax(model_out)==2:
    label="galloping"
if np.argmax(model_out)==3:
    label="jumping"
if np.argmax(model_out)==4:
    label="running"

#displaying the video along with the label

cam = cv2.VideoCapture(path) 

while(True): 
      
   
    
        ret,frame = cam.read() 
 
        if ret==True:             
            
                
            cv2.putText(frame,label,(20,20),cv2.FONT_HERSHEY_DUPLEX,1,(0,0,255))
            cv2.imshow("frame",frame)
            
 
            if cv2.waitKey(1) & 0xFF == ord('q'):
                 break
        else:
            break
       
cam.release() 
cv2.destroyAllWindows() 




[[4.0035187e-03 2.5989317e-05 5.2147180e-01 1.8390948e-02 4.5610774e-01]]
